In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import analytic_2D_function as a2D

plt.rcParams["font.family"] = "Times"
plt.rcParams.update({'font.size': 50})
plt.rcParams['xtick.major.pad']='12'
plt.rcParams['ytick.major.pad']='12'

In [ ]:
# Looped input. ============================================

# Number of sources to be tested.
nsrc=[300,400,500,700]
# Number of realisations to be tested.
N=[10,50,100]
# Number of repeats.
Nr=50

# Fixed input. =============================================

# Number of transit times in the time series.
L=100.0
# Frequency band [Hz].
freqmin=100.0e3
freqmax=500.0e3

In [ ]:
# Loop over everything.

ddtt_rel=np.zeros([len(N),len(nsrc),Nr])

for r in range(Nr):
    for i in range(len(nsrc)):
        for j in range(len(N)):
            
            t1=time.time()
            print('r=%d, nsrc=%d, N=%d' % (r,nsrc[i],N[j]))
            ddtt,ddtt_rel[j,i,r],dv=a2D.tt_error(nsrc=nsrc[i],N=N[j],L=L,freqmin=freqmin, freqmax=freqmax)
            print('absolute time shift error: %g ms' % ddtt )
            print('relative time shift error: %g percent' % ddtt_rel[j,i,r])
            print('velocity error: %g m/s' % dv)
            t2=time.time()
            print('elapsed time: %f s' % (t2-t1))
            print('-----------------------------------------------------------')
            

In [ ]:
# Compute average traveltime difference error.

ddtt_rel_avg=np.zeros([len(N),len(nsrc)])

for i in range(len(nsrc)):
    for j in range(len(N)):
        ddtt_rel_avg[j,i]=np.sum(np.abs(ddtt_rel[j,i,:]))/float(Nr)
        
    
# Plot relative traveltime errors.

fig,ax1=plt.subplots(1, figsize=(25,15))

for i in range(len(N)):
    
    c=0.85-0.85*float(i)/float(len(N))
    
    ax1.plot(nsrc,np.max(np.abs(ddtt_rel[i,:,:]),axis=1),'--',color=[c,c,c],linewidth=4)
    ax1.plot(nsrc,np.abs(ddtt_rel_avg[i,:]),color=[c,c,c],linewidth=10)
    ax1.text(1.01*nsrc[-1],np.abs(ddtt_rel_avg[i,-1]),str(N[i]))

ax1.set_xlabel('number of sources',labelpad=15)
ax1.set_ylabel('rel. time error [%]',labelpad=15)

# Make secondary axis for velocity error.

def tt2v(x): return 15.0*x
def v2tt(x): return x/15.0
secax=ax1.secondary_yaxis('right', functions=(tt2v, v2tt))
secax.set_ylabel('vel. error [m/s]',labelpad=15)

# Embellish figure.

ax1.set_xlim([0.9*nsrc[0],1.1*nsrc[-1]])
#ax1.set_ylim([0.0, 0.2])
plt.tight_layout()
plt.grid()
plt.show()

# Save figure and data.

#plt.savefig('ddtt1000kHz.png',format='png')
#np.save('ddtt_1000kHz.npy',ddtt_rel)